# Feature engineering
Nesse notebook vamos tratar os dados dos jogadores e criar features novas com bases nos dados que temos

In [2]:
from database.bq_connection import execute
import pandas as pd

In [14]:
query = """
    SELECT
        *
    FROM
        `puc-sp.fifa.player`
    WHERE
        player_tags IS NOT NULL
    LIMIT 1000;
"""

data = execute(query, "C:\\Users\\gugat\\fifa22-players-analysis\\database\\filekey.key")
data.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,dob,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,208722,https://sofifa.com/player/208722/sadio-mane/22...,S. Mané,Sadio Mané,LW,89,89,101000000.0,270000.0,1992-04-10,...,65+3,65+3,65+3,68+3,65+3,59+3,59+3,59+3,65+3,19+3
1,209331,https://sofifa.com/player/209331/mohamed-salah...,M. Salah,Mohamed Salah Ghaly,RW,89,89,101000000.0,270000.0,1992-06-15,...,68+3,68+3,68+3,71+3,67+3,58+3,58+3,58+3,67+3,22+3
2,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,1991-06-28,...,80+3,80+3,80+3,79+3,75+3,69+3,69+3,69+3,75+3,21+3
3,202652,https://sofifa.com/player/202652/raheem-sterli...,R. Sterling,Raheem Sterling,"LW, RW",88,89,107500000.0,290000.0,1994-12-08,...,64+3,64+3,64+3,69+3,65+3,55+3,55+3,55+3,65+3,21+3
4,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,1985-02-05,...,59+3,59+3,59+3,63+3,60+3,53+3,53+3,53+3,60+3,20+3


Nesse dataset temos apenas `dob` (data de nascimento) dos jogadores, vamos adicionar uma coluna com as idades dos jogadores

In [15]:
from datetime import date, datetime
current_date = datetime.today()

data.dob = pd.to_datetime(data.dob)

def age(dob):
    today = date.today()
    return today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))

data["age"] = data.dob.apply(age)
data[["dob", "age"]].head()

,dob,age
0,1992-04-10,30
1,1992-06-15,29
2,1991-06-28,30
3,1994-12-08,27
4,1985-02-05,37


Outra tratativa a ser feita é transformar cada `player_traits` e cada `player_tags` diferente em uma coluna boleana

In [16]:
data[["player_traits", "player_tags", "player_positions"]]

,player_traits,player_tags,player_positions
0,"Flair, Speed Dribbler (AI)","#Speedster, #Dribbler, #Acrobat, #Clinical Fin...",LW
1,"Finesse Shot, Long Shot Taker (AI), Speed Drib...","#Speedster, #Dribbler, #Acrobat, #Clinical Fin...",RW
2,"Injury Prone, Leadership, Early Crosser, Long ...","#Dribbler, #Playmaker, #Engine, #Distance Shoo...","CM, CAM"
3,"Flair, Speed Dribbler (AI)","#Speedster, #Dribbler, #Acrobat","LW, RW"
4,"Power Free-Kick, Flair, Long Shot Taker (AI), ...","#Aerial Threat, #Dribbler, #Distance Shooter, ...","ST, LW"
...,...,...,...
995,"Finesse Shot, Flair, Long Shot Taker (AI), Out...","#Dribbler, #FK Specialist","LW, CF"
996,"Flair, Long Shot Taker (AI), Outside Foot Shot...","#Dribbler, #Crosser, #FK Specialist, #Acrobat","RW, LW"
997,"Finesse Shot, Flair, Long Shot Taker (AI), Pla...",#Acrobat,"ST, LW, RW"
998,"Flair, Technical Dribbler (AI)","#Dribbler, #Acrobat","ST, RM"


In [17]:
player_traits_dummies = data.player_traits.str.get_dummies(sep=',')
player_tags_dummies = data.player_tags.str.get_dummies(sep=',')
player_positions_dummy = data.player_positions.str.get_dummies(sep=', ')

processed_data = pd.concat([data, player_traits_dummies, player_tags_dummies, player_positions_dummy], axis=1)
processed_data.drop(columns=["player_traits", "player_tags", "player_positions"], inplace=True)
processed_data.head()

,sofifa_id,player_url,short_name,long_name,overall,potential,value_eur,wage_eur,dob,weight_kg,...,CM,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
0,208722,https://sofifa.com/player/208722/sadio-mane/22...,S. Mané,Sadio Mané,89,89,101000000.0,270000.0,1992-04-10,69,...,0,0,0,1,0,0,0,0,0,0
1,209331,https://sofifa.com/player/209331/mohamed-salah...,M. Salah,Mohamed Salah Ghaly,89,89,101000000.0,270000.0,1992-06-15,71,...,0,0,0,0,0,0,0,1,0,0
2,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,91,91,125500000.0,350000.0,1991-06-28,70,...,1,0,0,0,0,0,0,0,0,0
3,202652,https://sofifa.com/player/202652/raheem-sterli...,R. Sterling,Raheem Sterling,88,89,107500000.0,290000.0,1994-12-08,69,...,0,0,0,1,0,0,0,1,0,0
4,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,91,91,45000000.0,270000.0,1985-02-05,83,...,0,0,0,1,0,0,0,0,0,1


,#Acrobat,#Aerial Threat,#Clinical Finisher,#Complete Defender,#Complete Forward,#Complete Midfielder,#Crosser,#Distance Shooter,#Dribbler,#Engine,...,#FK Specialist,#Playmaker,#Playmaker,#Poacher,#Speedster,#Strength,#Tackling,#Tackling,#Tactician,#Tactician
0,1,0,1,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,1,0,1,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,1,0,1,0,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
